In [1]:
!pip install pytorch-lightning

  Using cached pytorch_lightning-1.4.7-py3-none-any.whl (923 kB)
  Using cached torchmetrics-0.5.1-py3-none-any.whl (282 kB)
  Using cached pyDeprecate-0.3.1-py3-none-any.whl (10 kB)
  Using cached aiohttp-3.7.4.post0-cp37-cp37m-manylinux2014_x86_64.whl (1.3 MB)
  Using cached yarl-1.6.3-cp37-cp37m-manylinux2014_x86_64.whl (294 kB)
  Using cached async_timeout-3.0.1-py3-none-any.whl (8.2 kB)
  Using cached multidict-5.1.0-cp37-cp37m-manylinux2014_x86_64.whl (142 kB)


# Contents
1. model
2. optimizer
3. data
4. training loop
5. validation loop

In [1]:
%rm -rf lightning_logs
import torch
from torch import nn

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.tensorboard import SummaryWriter
from functools import partial

import model_fn

## Model

In [2]:
optimizer=partial(torch.optim.Adam,betas=(0.1,0.9),lr=1e-4)
loss_fn=torch.nn.CrossEntropyLoss()

In [3]:
net=nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

model=model_fn.MDL(net,optimizer=optimizer,loss_fn=loss_fn)

trainer=pl.Trainer(gpus=1,
                   progress_bar_refresh_rate=20,
                   max_epochs=3,
                  logger=TensorBoardLogger("lightning_logs/",version='try1',sub_dir="1"))
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/jovyan/.local/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name    | Type             | Params
---------------------------------------------
0 | net     | Sequential       | 136 K 
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
136 K     Trainable par

Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.functional.classification.accuracy.accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [4]:
trainer.test(model,model.val_dataloader())

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:99: UserWarning: you passed in a test_dataloader but have no test_step. Skipping test loop
  rank_zero_warn(f"you passed in a {loader_name} but have no {step_name}. Skipping {stage} loop")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


[]

In [ ]:
model.log()